This file will help predict the incoming ticket data as to which group it will fall

In [1]:
# import all necessary libraries

import os
import pandas as pd
import itertools
import numpy as np
import seaborn as sns
import pickle
import string
import math

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score
from sklearn.metrics import precision_score, recall_score, roc_curve, auc
from sklearn.utils import class_weight

from collections import OrderedDict
from gensim.models.word2vec import Word2Vec
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, load_model
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D, Input,Flatten
from keras.initializers import Constant
from keras.utils import np_utils

Using TensorFlow backend.


In [0]:
#Set your project path 
project_path =  '/content/drive/My Drive/Colab Notebooks/DataSets'

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
# set the google drive directory path to read and save all files there
os.chdir(project_path)

In [0]:
# taking groups which have ticket counts greater than 200 in one part and remaining in the other part

#L12 = ['GRP_0', 'GRP_8', 'GRP_24', 'GRP_12', 'GRP_9', 'GRP_2', 'GRP_19', 'GRP_3']
# L12 = ['GRP_0', 'GRP_8','GRP_9']
#L3 = [ 'GRP_1', 'GRP_4', 'GRP_5', 'GRP_6', 'GRP_7', 'GRP_10', 'GRP_11',  'GRP_13', 'GRP_14', 'GRP_15', 'GRP_16', 'GRP_17', 'GRP_18', 
#       'GRP_20', 'GRP_21', 'GRP_22', 'GRP_23',  'GRP_25', 'GRP_26', 'GRP_27', 'GRP_28', 'GRP_29', 'GRP_30', 'GRP_31',
#       'GRP_33', 'GRP_34', 'GRP_35', 'GRP_36', 'GRP_37', 'GRP_38', 'GRP_39', 'GRP_40', 'GRP_41', 'GRP_42', 'GRP_43', 'GRP_44',
#       'GRP_45', 'GRP_46', 'GRP_47', 'GRP_48', 'GRP_49', 'GRP_50',  'GRP_51', 'GRP_52', 'GRP_53', 'GRP_54', 'GRP_55', 'GRP_56',
#       'GRP_57', 'GRP_58', 'GRP_59', 'GRP_60', 'GRP_61', 'GRP_32',  'GRP_62', 'GRP_63', 'GRP_64', 'GRP_65',
#       'GRP_66', 'GRP_67', 'GRP_68', 'GRP_69', 'GRP_70', 'GRP_71', 'GRP_72', 'GRP_73']
L12 = ['GRP_0', 'GRP_8', 'GRP_24', 'GRP_12', 'GRP_9', 'GRP_2', 'GRP_19', 'GRP_3']
L3  = ['GRP_10', 'GRP_13', 'GRP_14','GRP_5','GRP_25','GRP_98','GRP_99']

In [0]:
vocabulary = dict()
inverse_vocabulary = ['<unk>']
embedding_dim = 300
embeddings = 1 * np.random.randn(30000, embedding_dim)
   
#colsToTrainOn =['CombinedDesc']
colsToTrainOn =['LemmaString']

In [0]:
def text_to_word_list(text):
  ''' Pre process and convert texts to a list of words '''
  text = str(text)
  text = text.lower().replace('.','').replace(',',' ').replace('-',' ').replace('  ', ' ')
  text = text.split()

  return text

Encode the two groups L12/L3 in 0 and 1

In [0]:
def SetGrp(text):
  ''' this will set the L12 to 0
  and L3 to 1'''
  
  if text in L12:
      return 0
  if text in L3:
      return 1

In [0]:
def combinedDesc(row):
  ''' to combine the short description and Description column together '''
  row['CombinedDesc'] = row['ShortDescSplitted'] + row['DescriptionSplitted']
  return row

In [0]:
#Remove punctuations
def remove_punctuations(text):
  ''' Remove all punctuations from the text column values '''
  for punctuation in string.punctuation:
      text = text.replace(punctuation, ' ')
  return text

In [0]:
# cant use stop words of nltk, as we have to retain some words which for a meaningful context to the ticket description
# so decided our own list of stop words
stop_words = []
stop_words += ["good", "evening", "night", "afternoon", "ca", "nt", "i", "vip", "llv", "xyz", "cid", "image", "gmail","co", "in", "com", "ticket", "company", "received", "0o", "0s", "3a", "3b", "3d", "6b", "6o", "a", "A", "a1", "a2", "a3", "a4", "ab", "able", "about", "above", "abst", "ac", "accordance", "according", "accordingly", "across", "act", "actually", "ad", "added", "adj", "ae", "af", "affected", "affecting", "after", "afterwards", "ag", "again", "against", "ah", "ain", "aj", "al", "all", "allow", "allows", "almost", "alone", "along", "already", "also", "although", "always", "am", "among", "amongst", "amoungst", "amount", "an", "and", "announce", "another", "any", "anybody", "anyhow", "anymore", "anyone", "anyway", "anyways", "anywhere", "ao", "ap", "apart", "apparently", "appreciate", "approximately", "ar", "are", "aren", "arent", "arise", "around", "as", "aside", "ask", "asking", "at", "au", "auth", "av", "available", "aw", "away", "awfully", "ax", "ay", "az", "b", "B", "b1", "b2", "b3", "ba", "back", "bc", "bd", "be", "became", "been", "before", "beforehand", "beginnings", "behind", "below", "beside", "besides", "best", "between", "beyond", "bi", "bill", "biol", "bj", "bk", "bl", "bn", "both", "bottom", "bp", "br", "brief", "briefly", "bs", "bt", "bu", "but", "bx", "by", "c", "C", "c1", "c2", "c3", "ca", "call", "came", "can", "cc", "cd", "ce", "certain", "certainly", "cf", "cg", "ch", "ci", "cit", "cj", "cl", "clearly", "cm", "cn", "co", "com", "come", "comes", "con", "concerning", "consequently", "consider", "considering", "could", "couldn", "couldnt", "course", "cp", "cq", "cr", "cry", "cs", "ct", "cu", "cv", "cx", "cy", "cz", "d", "D", "d2", "da", "date", "dc", "dd", "de", "definitely", "describe", "described", "despite", "detail", "df", "di", "did", "didn", "dj", "dk", "dl", "do", "does", "doesn", "doing", "don", "done", "down", "downwards", "dp", "dr", "ds", "dt", "du", "due", "during", "dx", "dy", "e", "E", "e2", "e3", "ea", "each", "ec", "ed", "edu", "ee", "ef", "eg", "ei", "eight", "eighty", "either", "ej", "el", "eleven", "else", "elsewhere", "em", "en", "end", "ending", "enough", "entirely", "eo", "ep", "eq", "er", "es", "especially", "est", "et", "et-al", "etc", "eu", "ev", "even", "ever", "every", "everybody", "everyone", "everything", "everywhere", "ex", "exactly", "example", "except", "ey", "f", "F", "f2", "fa", "far", "fc", "few", "ff", "fi", "fifteen", "fifth", "fify", "fill", "find", "fire", "five", "fix", "fj", "fl", "fn", "fo", "followed", "following", "follows", "for", "former", "formerly", "forth", "forty", "found", "four", "fr", "from", "front", "fs", "ft", "fu", "full", "further", "furthermore", "fy", "g", "G", "ga", "gave", "ge", "get", "gets", "getting", "gi", "give", "given", "gives", "giving", "gj", "gl", "go", "goes", "going", "gone", "got", "gotten", "gr", "greetings", "gs", "gy", "h", "H", "h2", "h3", "had", "hadn", "happens", "hardly", "has", "hasn", "hasnt", "have", "haven", "having", "he", "hed", "hello", "help", "hence", "here", "hereafter", "hereby", "herein", "heres", "hereupon", "hes", "hh", "hi", "hid", "hither", "hj", "ho", "hopefully", "how", "howbeit", "however", "hs", "http", "hu", "hundred", "hy", "i2", "i3", "i4", "i6", "i7", "i8", "ia", "ib", "ibid", "ic", "id", "ie", "if", "ig", "ignored", "ih", "ii", "ij", "il", "im", "immediately", "in", "inasmuch", "inc", "indeed", "index", "indicate", "indicated", "indicates", "information", "inner", "insofar", "instead", "interest", "into", "inward", "io", "ip", "iq", "ir", "is", "isn", "it", "itd", "its", "iv", "ix", "iy", "iz", "j", "J", "jj", "jr", "js", "jt", "ju", "just", "k", "K", "ke", "keep", "keeps", "kept", "kg", "kj", "km", "ko", "l", "L", "l2", "la", "largely", "last", "lately", "later", "latter", "latterly", "lb", "lc", "le", "least", "les", "less", "lest", "let", "lets", "lf", "like", "liked", "likely", "line", "little", "lj", "ll", "ln", "lo", "look", "looking", "looks", "los", "lr", "ls", "lt", "ltd", "m", "M", "m2", "ma", "made", "mainly", "make", "makes", "many", "may", "maybe", "me", "meantime", "meanwhile", "merely", "mg", "might", "mightn", "mill", "million", "mine", "miss", "ml", "mn", "mo", "more", "moreover", "most", "mostly", "move", "mr", "mrs", "ms", "mt", "mu", "much", "mug", "must", "mustn", "my", "n", "N", "n2", "na", "name", "namely", "nay", "nc", "nd", "ne", "near", "nearly", "necessarily", "neither", "nevertheless", "new", "next", "ng", "ni", "nine", "ninety", "nj", "nl", "nn", "nobody", "non", "none", "nonetheless", "noone", "normally", "nos", "noted", "novel", "now", "nowhere", "nr", "ns", "nt", "ny", "o", "O", "oa", "ob", "obtain", "obtained", "obviously", "oc", "od", "of", "off", "often", "og", "oh", "oi", "oj", "ok", "okay", "ol", "old", "om", "omitted", "on", "once", "one", "ones", "only", "onto", "oo", "op", "oq", "or", "ord", "os", "ot", "otherwise", "ou", "ought", "our", "out", "outside", "over", "overall", "ow", "owing", "own", "ox", "oz", "p", "P", "p1", "p2", "p3", "page", "pagecount", "pages", "par", "part", "particular", "particularly", "pas", "past", "pc", "pd", "pe", "per", "perhaps", "pf", "ph", "pi", "pj", "pk", "pl", "placed", "please", "plus", "pm", "pn", "po", "poorly", "pp", "pq", "pr", "predominantly", "presumably", "previously", "primarily", "probably", "promptly", "proud", "provides", "ps", "pt", "pu", "put", "py", "q", "Q", "qj", "qu", "que", "quickly", "quite", "qv", "r", "R", "r2", "ra", "ran", "rather", "rc", "rd", "re", "readily", "really", "reasonably", "recent", "recently", "ref", "refs", "regarding", "regardless", "regards", "related", "relatively", "research-articl", "respectively", "resulted", "resulting", "results", "rf", "rh", "ri", "right", "rj", "rl", "rm", "rn", "ro", "rq", "rr", "rs", "rt", "ru", "run", "rv", "ry", "s", "S", "s2", "sa", "said", "saw", "say", "saying", "says", "sc", "sd", "se", "sec", "second", "secondly", "section", "seem", "seemed", "seeming", "seems", "seen", "sent", "seven", "several", "sf", "shall", "shan", "shed", "shes", "show", "showed", "shown", "showns", "shows", "si", "side", "since", "sincere", "six", "sixty", "sj", "sl", "slightly", "sm", "sn", "so", "some", "somehow", "somethan", "sometime", "sometimes", "somewhat", "somewhere", "soon", "sorry", "sp", "specifically", "specified", "specify", "specifying", "sq", "sr", "ss", "st", "still", "stop", "strongly", "sub", "substantially", "successfully", "such", "sufficiently", "suggest", "sup", "sure", "sy", "sz", "t", "T", "t1", "t2", "t3", "take", "taken", "taking", "tb", "tc", "td", "te", "tell", "ten", "tends", "tf", "th", "than", "thank", "thanks", "thanx", "that", "thats", "the", "their", "theirs", "them", "themselves", "then", "thence", "there", "thereafter", "thereby", "thered", "therefore", "therein", "thereof", "therere", "theres", "thereto", "thereupon", "these", "they", "theyd", "theyre", "thickv", "thin", "think", "third", "this", "thorough", "thoroughly", "those", "thou", "though", "thoughh", "thousand", "three", "throug", "through", "throughout", "thru", "thus", "ti", "til", "tip", "tj", "tl", "tm", "tn", "to", "together", "too", "took", "top", "toward", "towards", "tp", "tq", "tr", "tried", "tries", "truly", "try", "trying", "ts", "tt", "tv", "twelve", "twenty", "twice", "two", "tx", "u", "U", "u201d", "ue", "ui", "uj", "uk", "um", "un", "under", "unfortunately", "unless", "unlike", "unlikely", "until", "unto", "uo", "up", "upon", "ups", "ur", "us", "used", "useful", "usefully", "usefulness", "using", "usually", "ut", "v", "V", "va", "various", "vd", "ve", "very", "via", "viz", "vj", "vo", "vol", "vols", "volumtype", "vq", "vs", "vt", "vu", "w", "W", "wa", "was", "wasn", "wasnt", "way", "we", "wed", "welcome", "well", "well-b", "went", "were", "weren", "werent", "what", "whatever", "whats", "when", "whence", "whenever", "where", "whereafter", "whereas", "whereby", "wherein", "wheres", "whereupon", "wherever", "whether", "which", "while", "whim", "whither", "who", "whod", "whoever", "whole", "whom", "whomever", "whos", "whose", "why", "wi", "widely", "with", "within", "without", "wo", "won", "wonder", "wont", "would", "wouldn", "wouldnt", "www", "x", "X", "x1", "x2", "x3", "xf", "xi", "xj", "xk", "xl", "xn", "xo", "xs", "xt", "xv", "xx", "y", "Y", "y2", "yes", "yet", "yj", "yl", "you", "youd", "your", "youre", "yours", "yr", "ys", "yt", "z", "Z", "zero", "zi", "zz"]

In [0]:
def removeString(data, regex):
    return data.str.lower().str.replace(regex.lower(), ' ')

In [0]:
def getRegexList():
  ''' This function will help to remove all the regex list that is defined below 
  from the text column for which it will be called '''
  
  regexList = []
  regexList += ['From:(.*)\r\n']  # from line
  regexList += ['ticket[_]*[\s]*[0-9]*'] # ticket id
  regexList += ['Sent:(.*)\r\n']  # sent to line
  regexList += ['Received:(.*)\r\n']  # received data line
  regexList += ['To:(.*)\r\n']  # to line
  regexList += ['CC:(.*)\r\n']  # cc line
  regexList += ['\[cid:(.*)]']  # images cid
  regexList += ['https?:[^\]\n\r]+']  # https & http
  regexList += ['Subject:']
  regexList += ['[\w\d\-\_\.]+@[\w\d\-\_\.]+']  # emails
  regexList += ['[0-9][\-0–90-9 ]+']  # phones
  regexList += ['[0-9]']  # numbers
  regexList += ['[^a-zA-z 0-9]+']  # anything that is not a letter
  regexList += ['[\r\n]']  # \r\n
  regexList += [' [a-zA-Z] ']  # single letters
  regexList += ["  "]  # double spaces
  regexList += ['^[_a-z0-9-]+(\.[_a-z0-9-]+)*@[a-z0-9-]+(\.[a-z0-9-]+)*(\.[a-z]{2,4})$']
  regexList += ['[\w\d\-\_\.]+ @ [\w\d\-\_\.]+']
  regexList += ['Subject:']
  regexList += ['[^a-zA-Z]']

  return regexList

In [0]:
def readFile():    
  ''' This function will read the file that is saved after the EDA completion
  Will drop NA values which contitute only 9 rows in total
  will apply the normal cleaning methods of removing dupliates, removing some regex list, and punctuations from the text columns
  Will convert text to word list
  Combine to columns into one
  Set Level Groups to 0 or 1 '''


#  df = pd.read_excel('TicketAssignments.xlsx')
  df = pd.read_excel('TestDataToPredict_Description.xlsx')
  df.dropna(subset=['Short description', 'Description'], inplace=True)
  
  df['Short description'] = (df['Short description'].str.split().apply(lambda x: OrderedDict.fromkeys(x).keys()).str.join(' '))
  df['Short description'] = df['Short description'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
  df['Short description'] = df['Short description'].apply(remove_punctuations)
  df['Description'] = (df['Description'].str.split().apply(lambda x: OrderedDict.fromkeys(x).keys()).str.join(' '))
  df['Description'] = df['Description'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))
  df['Description'] = df['Description'].apply(remove_punctuations)

  for regex in getRegexList():
    df['Description'] = removeString(df['Description'], regex)
    
  df['ShortDescSplitted'] = df['Short description'].apply(lambda x : text_to_word_list(x))
  df['DescriptionSplitted'] = df['Description'].apply(lambda x : text_to_word_list(x))
  
  # first assign unknown and then will fill the data
  df['CombinedDesc'] = 'UNK9999'
  df = df.apply(lambda x :combinedDesc(x), axis=1 )
  df['CombinedDescRetained'] = df['CombinedDesc']
  df['Level1Grp'] = 0
  df['Level1Grp'] = df['Assignment group'].apply(lambda x : SetGrp(x))
  df['Assignmentgroup'] = df['Assignment group']

  return df

In [0]:
def PrepDataForPrediction(df,vocabulary,inverse_vocabulary):
  ''' Will create the vocabulary and inverse_vocabulary list to be used in embedding 
  and later to be referred when we will call the same in our predict section '''
  
  for dataset in [df]:
    for index, row in dataset.iterrows():

      # Iterate through the text of description column of the row
      for question in colsToTrainOn:
        q2n = []  
        for word in row[question]:
          if word not in vocabulary:
              vocabulary[word] = len(inverse_vocabulary)
              q2n.append(len(inverse_vocabulary))
              inverse_vocabulary.append(word)
          else:
              q2n.append(vocabulary[word])
            # Replace description as word to description as number representation
          dataset.at[index, question]= q2n
  return df, vocabulary,inverse_vocabulary

In [0]:
def PrepDataForModel(df, maxlen):
  X = df[colsToTrainOn[0]]
  X = pad_sequences(X, maxlen=maxlen,truncating='post',padding='post',value=0)

  return X

In [0]:
def PrepDataForModel2_Grp0(df,max_seq_length):
    
  partDfZero = df.loc[df['Level1Grp'] == 0]
  partDfOne = df.loc[df['Level1Grp'] == 1]

  le = LabelEncoder()

  partDfZero['Assignmentgroup'] = le.fit_transform(partDfZero['Assignment group'])
  partDfOne['Assignmentgroup'] = le.fit_transform(partDfOne['Assignment group'])


  X_Zero = partDfZero[colsToTrainOn[0]]
  Y_Zero = np_utils.to_categorical(partDfZero['Assignmentgroup'])

  X_One = partDfOne[colsToTrainOn[0]]
  Y_One = np_utils.to_categorical(partDfOne['Assignmentgroup'])
      
  X_train_Zero, X_validation_Zero, Y_train_Zero, Y_validation_Zero = train_test_split(X_Zero, Y_Zero, test_size=math.floor(len(X_Zero)*0.2))
  X_train_One, X_validation_One, Y_train_One, Y_validation_One = train_test_split(X_One, Y_One, test_size=math.floor(len(X_One)*0.2))

  X_train_Zero = pad_sequences(X_train_Zero, maxlen=maxlen,truncating='post',padding='post',value=0)
  X_validation_Zero = pad_sequences(X_validation_Zero, maxlen=maxlen,truncating='post',padding='post',value=0)

  X_train_One = pad_sequences(X_train_One, maxlen=maxlen,truncating='post',padding='post',value=0)
  X_validation_One = pad_sequences(X_validation_One, maxlen=maxlen,truncating='post',padding='post',value=0)


  return X_train_Zero, Y_train_Zero, X_validation_Zero,Y_validation_Zero,X_train_One,X_validation_One,Y_train_One,Y_validation_One,partDfZero,partDfOne

In [0]:
def PrepDataForModel_2(df, maxlen):
  partDfZero = df.loc[df['PredictedClassLabel'] == 0]
  partDfOne = df.loc[df['PredictedClassLabel'] == 1]

  X_Zero = partDfZero[colsToTrainOn[0]]
  X_Zero = pad_sequences(X_Zero, maxlen=maxlen,truncating='post',padding='post',value=0)


  X_One = partDfOne[colsToTrainOn[0]]
  X_One = pad_sequences(X_One, maxlen=maxlen,truncating='post',padding='post',value=0)

  return X_Zero, X_One,partDfZero,partDfOne

In [0]:
def BuildEmbeddingMatrix(word2vec,embeddings):
  count = 0    
  for word, index in vocabulary.items():
    if word in word2vec.vocab:
      count+=1
      embeddings[index] = word2vec.word_vec(word)
    else:
      embeddings[index] = 0

  return embeddings

In [0]:
def ReadPickeledObjects():
  with open('word2vec.pickle', 'rb') as f:
    word2vec = pickle.load(f)
      
  with open('embeddings.pickle', 'rb') as f:
    embeddings = pickle.load(f)
      
  with open('vocabulary.pickle', 'rb') as f:
    vocabulary = pickle.load(f)
      
  with open('inverse_vocabulary.pickle', 'rb') as f:
    inverse_vocabulary = pickle.load(f)
      
  return word2vec,embeddings,vocabulary,inverse_vocabulary

In [21]:
df = pd.read_excel('TestDataToPredict.xlsx')
df.head()

,LemmaString,NewAssignmentGroup
0,outlook team meeting skype not appear calendar...,GRP_0
1,user want reset password,GRP_0
2,engineer tool not connect unable submit report,GRP_0
3,duplication network address gentles device sha...,GRP_98
4,job fail scheduler monitor tool,GRP_8


In [22]:
  df['Level1Grp'] = 0
  df['Level1Grp'] = df['NewAssignmentGroup'].apply(lambda x : SetGrp(x))
  #df['Assignmentgroup'] = df['NewAssignmentGroup']
  df.head()

,LemmaString,NewAssignmentGroup,Level1Grp
0,outlook team meeting skype not appear calendar...,GRP_0,0
1,user want reset password,GRP_0,0
2,engineer tool not connect unable submit report,GRP_0,0
3,duplication network address gentles device sha...,GRP_98,1
4,job fail scheduler monitor tool,GRP_8,0


In [23]:
#df = readFile()
#maxlen = min(df['CombinedDesc'].map(lambda x:len(x)).max(), 150)
maxlen = min(df['LemmaString'].map(lambda x:len(x)).max(), 150)
print(maxlen)

150


In [0]:
word2vec,embeddings,vocabulary,inverse_vocabulary = ReadPickeledObjects() 

df, vocabulary,inverse_vocabulary= PrepDataForPrediction(df,vocabulary,inverse_vocabulary)
  
embeddings = BuildEmbeddingMatrix(word2vec,embeddings)

In [25]:
#df['CombinedDesc'].map(lambda x:len(x)).max()
df['LemmaString'].map(lambda x:len(x)).max()

155

In [0]:
# maxlen=150

In [0]:
import tensorflow as tf
from keras.models import load_model

In [27]:
mode_FirstLayer= tf.keras.models.load_model('model_FirstLevelGrouping.h5')
mode_SecondLayerZeroGroup= tf.keras.models.load_model('model_L12_AssignmentGroups.h5')
mode_SecondLayerFirstGroup= tf.keras.models.load_model('model_L3_AssignmentGroups.h5')

In [0]:
# First Model Preidction
X = PrepDataForModel(df, maxlen)
y = mode_FirstLayer.predict(X)
predictedClass = np.argmax(y,axis=1).tolist()
df['PredictedClassLabel'] = pd.Series(predictedClass)

In [29]:
print(X.shape)
X

(9, 150)


array([[ 2,  8, 15, ...,  0,  0,  0],
       [ 8,  7,  9, ...,  0,  0,  0],
       [ 9,  5,  3, ...,  0,  0,  0],
       ...,
       [ 1, 16, 23, ...,  0,  0,  0],
       [ 5,  9,  9, ...,  0,  0,  0],
       [ 9, 11, 18, ...,  0,  0,  0]], dtype=int32)

In [30]:
print(y.shape)
y

(9, 2)


array([[9.9491113e-01, 4.4445507e-03],
       [9.4871271e-01, 4.3404773e-02],
       [9.9516368e-01, 4.3767062e-03],
       [9.9370390e-01, 6.9068917e-03],
       [9.9964881e-01, 4.0190705e-04],
       [4.6511171e-03, 9.9473971e-01],
       [7.5164503e-01, 2.3509455e-01],
       [4.2601645e-01, 5.2969080e-01],
       [4.1483190e-02, 9.5925528e-01]], dtype=float32)

In [31]:
df.head()

,LemmaString,NewAssignmentGroup,Level1Grp,PredictedClassLabel
0,"[2, 8, 15, 1, 2, 2, 21, 6, 15, 9, 16, 17, 6, 1...",GRP_0,0,0
1,"[8, 7, 9, 11, 6, 22, 16, 5, 15, 6, 11, 9, 7, 9...",GRP_0,0,0
2,"[9, 5, 3, 4, 5, 9, 9, 11, 6, 15, 2, 2, 1, 6, 5...",GRP_0,0,0
3,"[14, 8, 18, 1, 4, 19, 16, 15, 4, 2, 5, 6, 5, 9...",GRP_98,1,0
4,"[26, 2, 23, 6, 12, 16, 4, 1, 6, 7, 19, 20, 9, ...",GRP_8,0,0


In [0]:
X_Zero, X_One,partDfZero,partDfOne = PrepDataForModel_2(df, maxlen)

In [33]:
print(X_Zero.shape)
print(X_One.shape)
print(partDfZero.shape)
print(partDfOne.shape)

(6, 150)
(3, 150)
(6, 4)
(3, 4)


In [0]:
y = mode_SecondLayerZeroGroup.predict(X_Zero)
predictedClassGroupZero = np.argmax(y,axis=1).tolist()

In [35]:
X_One

array([[11,  9,  7,  9, 15,  6, 18, 16,  7,  7, 22,  2, 11, 14,  6, 20,
        14, 15, 20, 13,  6, 20,  7, 15, 14, 14,  6, 18, 16,  7,  7, 22,
         2, 11, 14,  6, 17, 16,  5, 16,  3,  9, 17,  9,  5, 15,  6, 15,
         2,  2,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0],
       [ 5,  9,  9, 14,  6, 22,  4, 11,  9,  1,  9,  7,  7,  6, 17,  2,
         8,  7,  9,  6,  1, 16, 18, 15,  2, 18,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,

In [0]:
y = mode_SecondLayerFirstGroup.predict(X_One)
predictedClassGroupOne = np.argmax(y,axis=1).tolist()

In [0]:
# predictedClassGroupOne

In [0]:
df['PredictedGroup'] = 'UNK9999'

In [0]:
count=0
for index in partDfZero.index:
  x = predictedClassGroupZero[count]
  df.at[index,'PredictedGroup'] = L12[x]
  count+=1

In [0]:
count=0
for index in partDfOne.index:
  x = predictedClassGroupOne[count]
  df.at[index,'PredictedGroup'] = L3[x]
  count+=1

In [0]:
df.to_excel('Output.xlsx', index=False)